<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/food_habits_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food habits analytics

## Uploading packages and data

In [1]:
#Importing necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
#Remove previous versions of the uploaded excel file
!rm food_habits.xlsx

In [ ]:
#Uploading file from local drive
from google.colab import files
uploaded1 = files.upload()

In [4]:
#Storing dataset in a Pandas Dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['food_habits.xlsx'])) #==> setting Event (datetime) as index to perform date slicing

In [5]:
#Checking the dataframe info
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        78 non-null     datetime64[ns]
 1   time        78 non-null     object        
 2   food        70 non-null     object        
 3   large_meal  78 non-null     int64         
 4   void_ev     78 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 3.2+ KB


##Data Manipulation

In [41]:
#Transforming to list
list_1 = df1['food'].str.cat(sep=', ').split(", ")
list_1[:10]

['bread',
 'pepperoni',
 'ham',
 'cheese',
 'ricotta',
 'mustard',
 'hot sauce',
 'potato chips',
 'pizza',
 'tomato_sauce']

In [46]:
#Building dictionary of occurrences for each food
import collections
occurrences = collections.Counter(list_1)
occurrences

Counter({'aioli': 13,
         'aoili': 1,
         'appel_ cider_vinager': 1,
         'apple': 5,
         'apple_cider_vinager': 1,
         'apple_cyder': 1,
         'arepa': 5,
         'avocado': 5,
         'bacon': 2,
         'baguette': 9,
         'balsamic': 5,
         'banana': 6,
         'bbq_sauce': 4,
         'beans': 4,
         'beef': 3,
         'beer': 3,
         'blue_cheese': 1,
         'bread': 23,
         'bread_sticks': 1,
         'brioche_bread': 2,
         'broccoli': 5,
         'brown_cheese': 1,
         'burger': 2,
         'butter': 4,
         'butter_cookie': 1,
         'cake': 6,
         'calzone': 1,
         'carrot': 9,
         'carrots': 16,
         'cauliflower': 1,
         'celery': 9,
         'cereal': 1,
         'champignon': 9,
         'champignon_sauce': 1,
         'cheddar': 1,
         'cheese': 42,
         'cheese_balls': 5,
         'cheese_cake': 6,
         'cheez_doodles': 1,
         'chia': 1,
         'chicken'

In [42]:
#Transforming list of foods into series
foods_1 = pd.Series(data=list_1)
foods_1.tail()

1029           bread
1030           bread
1031    salt_cookies
1032        oat_milk
1033         granola
dtype: object

In [43]:
#Counting most popular foods
foods_1.value_counts()

cheese                  42
chocolate               37
rice                    37
onion                   30
soda                    27
                        ..
pastry_cream             1
garlic_podwer            1
frozen_pizza             1
chicken_cordon_blue      1
appel_ cider_vinager     1
Length: 252, dtype: int64

In [14]:
#Exporting to excel into local disk
from google.colab import files
df1.to_excel('food_habits.xlsx', index=False) #==> Excluding index from file
files.download('food_habits.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>